In [ ]:
import pandas as pd
import numpy as np
import os

## Clicks & Impressions (Relationships)

In [ ]:
# read the impressions from training only
col_names = ['impressionId', 'userId', 'timeString', 'history', 'impressions']
impression_df_train = pd.read_csv('./data/train/behaviors.tsv', sep = '\t', header = None, names = col_names)
impression_df_validate = pd.read_csv('./data/validate/behaviors.tsv', sep = '\t', header = None, names = col_names)

impression_df_train['splitSet'] = 'TRAIN'
impression_df_validate['splitSet'] = 'VALIDATION'
impression_df = pd.concat([impression_df_train, impression_df_validate], ignore_index=True)
#impression_df 

In [ ]:
impression_df['time'] = pd.to_datetime(impression_df.timeString, infer_datetime_format=True)
#impression_df

In [ ]:
def listify(x):
    res = None
    if not pd.isna(x):
        res = x.split(' ')
    return res

impression_df['historyList'] = impression_df.history.apply(listify)
impression_df['impressionList'] = impression_df.impressions.apply(listify)
#impression_df

In [ ]:
# Q is hustory consistently included or are earlier clicks cut off?
historic_df = impression_df.groupby('userId').agg({'impressionId': 'count', 'splitSet':'min', 
                                                   'history': lambda x: set(list(x))}) \
    .reset_index().sort_values(by = 'impressionId', ascending=False)
historic_df['numUniqueHistories'] = historic_df.history.apply(len)
historic_df[historic_df.numUniqueHistories > 1] 
#interestingly all impression histories are the same for each user. 

In [ ]:
def listify_from_set(x):
    return ' '.join(filter(lambda i: not pd.isnull(i), x)).split(' ')
historic_df['historyList'] = historic_df.history.apply(listify_from_set)
historic_df.drop(columns = ['impressionId', 'history', 'numUniqueHistories'], inplace=True)
historic_df = historic_df.explode('historyList')
historic_df.rename(columns = {'historyList':'newsId'}, inplace=True)
historic_df 

In [ ]:
impression_click_df = impression_df.loc[~impression_df.impressionList.isnull(),
    ['impressionId', 'userId', 'time', 'impressionList', 'splitSet']] \
    .explode('impressionList')
impression_click_df.rename(columns = {'impressionList':'newsIdHit'}, inplace=True)
impression_click_df[["newsId", "userClickedCode"]] = impression_click_df.newsIdHit.str.split('-', expand=True)
impression_click_df.drop(columns = ['newsIdHit'], inplace=True)
impression_click_df

In [ ]:
clicked_df = impression_click_df[impression_click_df.userClickedCode == '1'].drop(columns = ['userClickedCode'])
did_not_click_df = impression_click_df[impression_click_df.userClickedCode == '0'].drop(columns = ['userClickedCode'])


In [ ]:
output_path = './data/clean'
if not os.path.exists(output_path):
    os.mkdir(output_path)
    print(f'Created new directory: {output_path}')
else:
    print(f'Directory {output_path} already exists')

In [ ]:
historic_df.to_csv(f'{output_path}/historic-clicks.csv', index = False)
clicked_df.to_csv(f'{output_path}/clicks.csv', index = False)
did_not_click_df.to_csv(f'{output_path}/did-not-click.csv', index = False)

## Adding Users

In [ ]:
user_df = impression_df[['userId']].drop_duplicates()
user_df

In [ ]:
user_df.to_csv(f'{output_path}/users.csv', index = False)

## News

In [ ]:
col_names = ['newsId', 'category', 'subCategory', 'title', 'abstract', 'url', 'titleEntities','abstractEntites']
news_train_df = pd.read_csv('./data/train/news.tsv', sep = '\t', header = None, names = col_names)
news_train_df.drop(columns = ['titleEntities', 'abstractEntites'], inplace=True)
news_validate_df = pd.read_csv('./data/validate/news.tsv', sep = '\t', header = None, names = col_names)
news_validate_df.drop(columns = ['titleEntities', 'abstractEntites'], inplace=True)

In [ ]:
news_df = pd.concat([news_train_df, news_validate_df],ignore_index=True).drop_duplicates(subset='newsId')
news_df

In [ ]:
news_df.loc[news_df.abstract.isna(), 'abstract'] = ''

In [ ]:
#news_df.abstract = news_df.abstract.str.replace(',', ' -')
news_df.abstract = news_df.abstract.str.replace('"', "'")

#news_df.title = news_df.title.str.replace(',', ' -')
news_df.title = news_df.title.str.replace('"', "'")

In [ ]:
# a couple manual fixes
news_df.loc[news_df.newsId == 'N18259', 'abstract'] = "He'll be in better shape physically and he'll be in a better place mentally too,'' coach Freddie Kitchens said of Mayfield"
news_df.loc[news_df.newsId == 'N18259', 'url'] = 'https://assets.msn.com/labs/mind/AAIPbTZ.html'

In [ ]:
# a couple manual fixes
news_df.loc[news_df.newsId == 'N16590', 'abstract'] = "We're seeing really disturbing things coming out of the transcripts... the most disturbing I thought, aside from the open extortion of a foreign government, was the fact that the secretary of state had to call Sean Hannity to find out how the president was feeling about an ambassador, Jess McIntosh says. Tara Dowdell and Max Boot join the conversation."
news_df.loc[news_df.newsId == 'N16590', 'url'] = 'https://assets.msn.com/labs/mind/BBWymjv.html'



In [ ]:
news_df.to_csv(f'{output_path}/news.csv', index = False)